# Moment sequences supported in a semi-algebraic set and $L^p(\mathbf{K})$ density

In [8]:
%display typeset

$\def\S{\mathbf{S}}$
$\def\A{\mathbf{A}}$
$\def\b{\mathbf{b}}$
$\def\B{\mathbf{B}}$
$\def\K{\mathbf{K}}$
$\def\G{\mathbf{G}}$
$\def\S{\mathbf{S}}$
$\def\V{\mathbf{V}}$
$\def\X{\mathbf{X}}$
$\def\Y{\mathbf{Y}}$
$\def\x{\mathbf{x}}$
$\def\y{\mathbf{y}}$
$\def\p{\mathbf{p}}$
$\def\z{\mathbf{z}}$
$\def\M{\mathbf{M}}$
$\def\Q{\mathbf{Q}}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\N}{\mathbb{N}}$
$\newcommand{\vol}{\mathrm{vol}}$

**Contents:**
1. [Introduction](#1.-Introduction)
2. [Computation of $z_{q,\alpha}$](#2.-Computation-of-$z_{q,\alpha}$)
3. [$L^q$ norm of a polynomial](#3.-$L^q$-norm-of-a-polynomial)

**Keywords / SageMath :**

- numerical integrals
- exact integration over `RealSets` and `Polyhedron` objects
- univariate and multivariate polynomials

## 1. Introduction

Consider the polynomial $f \in \R[\x]$, that we write as the finite sum 
$f(x) = \sum_\alpha f_\alpha \x^\alpha$, for some set of coefficients $(f_\alpha)$ indexed by a multi-index $\alpha \in \N^n$. Here $\x^\alpha := x_1^{\alpha_1}\cdots x_n^{\alpha_n}$.

Let there be given an (infinite) sequence of real numbers $\y = (y_\alpha)$. Consider the linear functional $\ell_\y : \R[\x] \to \R$, which maps $f$ to $\ell_\y(f) = \sum_\alpha f_\alpha y_\alpha$, for any $f \in \R[\x]$. 

**Remark.**
This linear functional has the property that if the numerical sequence has an associated representing *measure*, then the application of the functional corresponds to integration with respect to the measure. Indeed, if $\y$ is the moment sequence of Borel measure $\mu$ supported on a compact basic semi-algebraic set $\K$, then
$$
y_\alpha = \int_\K x^\alpha d\mu(\x),
$$
and in consequence for all polynomials $\R[\x]$,
$$
\ell_\y(f) = \sum_\alpha f_\alpha y_\alpha = \sum_\alpha \int_\K  f_\alpha x^\alpha d\mu(\x) = \int_\K f(x)d\mu(\x).\qquad \square
$$

In the sequel and for the sake of simplicity (and the experiments), we stick to the one-dimensional case $n=1$. We build an infinite sequence of real numbers $\z_q = (z_{q,\alpha})$ as follows. 

First, let $q \in [1, \infty]$ be given, and let $p = q/(q-1)$ be its conjugate number (they satisfy $1/p + 1/q = 1$). It also lives in $[1,\infty]$, by definition. Let $\K$ be a basic compact semi-algebraic set. For simplicity, we will often consider $\K = [0, 1]$. Let the sequence $\z_q = (z_{q,\alpha})$ be defined as
$$
z_{q, \alpha} := \left(\int_\K x^{\alpha q} dx\right)^{1/q}.
$$

With these definitions, we can consider the linear functional as before:

- $\ell_{\z_q}(f) = \sum_\alpha f_\alpha z_{q,\alpha}$, for any $f \in \R[x]$,
- in the particular case $q=1$, one recovers the moments of the standard Lebesgue measure and $\ell_\z(f) = \sum_\alpha f_\alpha z_\alpha$, for any $f \in \R[x]$ is the usual integral of $f$.

**Remark.** Notice that for $q>1$ the moments are in general complex, see the example below. 

In [3]:
# Example: integral in K = [0, 1] for q = 3/2 (corresponding to p = 2)
integrate(exp(3/2*log(x)), x, -1, 1)

-2/5*I + 2/5

For this reason and for simplicity, if possible we scale the dynamics to the first orthant, or just $[0,1]^n$. In the latter case, one has the formula
$$
z_{q, \alpha} = \left(\int_{\K=[0,1]^n} x_1^{\alpha_1 q}\cdots x_n^{\alpha_n q} dx\right)^{1/q} =
\prod_{i=1}^n \frac{1}{(1+\alpha_i q)^{1/q}}.
$$

## 2. Computation of $z_{q,\alpha}$

If $\K = [0, 1]^n$ we use the analytic formula of above, but what if $\K$ is slighly more general set? (eg. a polytope). In this case, an analytic formula is often not available.

To compute the moment integrals $z_{q,\alpha}$, Sage provides two obvious ways:

- Numerical (inexact) computation using `numerical_integral` (using an adaptive integration algorithm).
- Exact computation of a polynomial over a polytope (using [LattE integrale](https://www.math.ucdavis.edu/~latte/software.php)).

The issue with the 2nd option is that it only makes sense when $\alpha q$ is an integer. 

#### Numerical computation

In [28]:
K = RealSet([0, 1])[0]
q = 3
alpha = 1
numerical_integral(x^(alpha*q), K.lower(), K.upper())[0]^(1/q)

0.6299605249474366

#### Exact computation

In [29]:
x = polygen(QQ, 'x')
x_alphaq = lambda alpha, q : x^(alpha*q)
y = x_alphaq(alpha=1, q=3)

In [32]:
Polyhedron(vertices=[[0], [1]]).integrate(y)^(1/q)

(1/4)^(1/3)

In [33]:
_.n()

0.629960524947437

But if we use $q=3/2$ (corresponding to $p=2$) then the exact computation fails (as expected), while the inexact computation gives:

In [34]:
x = var('x')
q = 3/2; alpha = 1
numerical_integral(x^(alpha*q), K.lower(), K.upper())[0]^(1/q)

0.5428835233063709

It corresponds to the analytic solution,

In [37]:
(1+alpha*q)^(-1/q)

2/5*(5/2)^(1/3)

In [38]:
_.n()

0.542883523318981

#### Provide both approaches as a keyworded argument

In [465]:
def moments_Zq(K=None, d=4, q=1, algorithm='numerical'):
    
    if K is None:
        if algorithm == 'numerical':
            # unit interval
            K = RealSet([0, 1])[0]
        elif algorithm == 'exact':
            # unit polyhedron
            K = Polyhedron(vertices=[[0], [1]])
            
    if algorithm == 'numerical':
        # assuming K is a RealSet
        x = SR.var('x')
        xmin = K.lower(); xmax = K.upper()
        x_alpha = []
        for alpha in range(d):
            x_alpha += [numerical_integral(x^(alpha*q), xmin, xmax)[0]^(1/q)]
    
    elif algorithm == 'exact':
        # assuming K is a Polyhedron
        if not q.is_integer():
            raise ValueError("The exact algorithm requires integer q")
        x = polygen(QQ, 'x')
        x_alphaq = lambda alpha, q : x^(alpha*q)
        x_alpha = []
        for alphai in range(d):
            y = x_alphaq(alpha=alphai, q=q)
            x_alpha += [K.integrate(y)^(1/q)]
    
    return x_alpha

#### Some Examples:

In [466]:
moments_Zq(RealSet([0, 1])[0], algorithm='numerical')

[1.0, 0.5, 0.3333333333333333, 0.25]

In [467]:
moments_Zq(Polyhedron(vertices=[[0], [1]]), algorithm='exact')

[1, 1/2, 1/3, 1/4]

In [468]:
moments_Zq(RealSet([0, 1])[0], 5, 3, algorithm='numerical')

[1.0,
 0.6299605249474366,
 0.5227579585747102,
 0.464158883361278,
 0.4252903702829902]

In [469]:
moments_Zq(Polyhedron(vertices=[[0], [1]]), 5, 3, algorithm='exact')

[1, (1/4)^(1/3), (1/7)^(1/3), (1/10)^(1/3), (1/13)^(1/3)]

## 3. $L^q$ norm of a polynomial

We seek to evaluate: 

- (i) $\ell_{\z_1}(|f|^q)^{1/q} = \left( \int_\K |f|^q d\lambda(x)\right)^{1/q} = \Vert f \Vert_q$, which abusing notation is $\left( \sum_\alpha z_{1, \alpha} |\sum_\alpha f_\alpha x^\alpha|^q\right)^{1/q}$. 
- (ii) $\sum_{\alpha} |f_\alpha| z_{\alpha, q}$

for polynomials $f \in \R[x]$.

By the triangle inequality, we have
$$
\left|\ell_{\z_q}(f)\right| = \left| \sum_{\alpha} f_\alpha z_{\alpha, q} \right| \leq \sum_{\alpha} |f_\alpha| z_{\alpha, q}.
$$

**Rappel** 
- *Holder's inequality* : $\Vert fg \Vert_1 \leq \Vert f\Vert_p \Vert g\Vert_q$
- *reverse Holder's inequality* : $\Vert fg \Vert_1 \geq \Vert f\Vert_{1/p} \Vert g\Vert_{-1/(p-1)}$
- *Lp embedding* : let $0< p < q \leq \infty$, then $\Vert f \Vert_p \leq \mu(X)^{1/p-1/q}\Vert f \Vert_q$. 
- the last one implies that if the measure of $X$ is finite, then $L^q \subset L^1$ for any $q \geq 1$.
$\square$

Using reverse Holder's inequality,
$$
\Vert f \Vert_q \geq \dfrac{1}{\vol(\K)^{1-1/q}} \sum_\alpha |f_\alpha| z_{\alpha, 1}.
$$

But how does it compare to $\left|\ell_{\z_q}(f)\right|$?

### Experiments

#### Linear functional $\ell_\y(f)$

Let write compute the linear functional with Sage.

In [427]:
# independent variable
x, = polygens(SR, 'x')

# coefficients (symbolic)
f0, f1, f2 = SR.var(['f0', 'f1', 'f2'])

f = f0 + f1*x + f2*x^2

In [428]:
# compute moments in [0, 1] for q = 3, up to order d = 8
qmoments_degree_eight_q_three = moments_Zq(q=3, d=8, algorithm='exact')
qmoments_degree_eight_q_three

[(1/4)^(1/3),
 (1/7)^(1/3),
 (1/10)^(1/3),
 (1/13)^(1/3),
 1/2*(1/2)^(1/3),
 (1/19)^(1/3),
 (1/22)^(1/3),
 (1/25)^(1/3)]

Let us compute $\ell_{\z_q}(f) = \sum_\alpha f_\alpha z_{\alpha, q}$.

In [429]:
def Ly(y, f, abs_elementwise=False):
    
    sum_abs_falpha = 0
    
    coeffs = f.coefficients()
    
    if abs_elementwise:
        coeffs = [abs(fi) for fi in coeffs]
        
    for i, falpha in enumerate(coeffs):
        z_alpha_q = y[i]
        sum_abs_falpha += falpha * z_alpha_q
    return sum_abs_falpha

In [430]:
Ly(qmoments_degree_eight_q_three, f)

(1/4)^(1/3)*f0 + (1/7)^(1/3)*f1 + (1/10)^(1/3)*f2

We have added the possibility of taking the elementwise absolute value of $f$, thus evaluating $\sum_\alpha |f_\alpha| z_{\alpha, q}$. For example,

In [431]:
Ly(qmoments_degree_eight_q_three, f, abs_elementwise=True)

(1/4)^(1/3)*abs(f0) + (1/7)^(1/3)*abs(f1) + (1/10)^(1/3)*abs(f2)

#### Lebesgue integral of $f \in \R[x]$ in the $q$ norm

Consider $\Vert f \Vert_q = \Vert f_0 + f_1 x\Vert_q$ over $\K$. As before, there are two ways:

- To compute the integral numerically we shall fix values for the coefficients.
- Compute the integral symbolically.

In [411]:
def norm_q(f, K=None, q=1, algorithm='numerical'):
    r"""Compute the q-norm of a polynomial over a polytope.
    
    TESTS::
    
    By default, use numerical algorithm::
    
        sage: norm_q(x)
        0.5
        sage: norm_q(x^3)
        0.25
        
    We can make exact computations with even exponent as well::
    
        sage: norm_q(x, q=2, algorithm='exact')
        sqrt(1/3)
        sage: norm_q(1 - 2*x^3 + x^4, q=4, algorithm='exact')
        (240167/510510)^(1/4)
    """
    if K is None:
        if algorithm == 'numerical':
            # unit interval
            K = RealSet([0, 1])[0]
        elif algorithm == 'exact':
            # unit polyhedron
            K = Polyhedron(vertices=[[0], [1]])
            
    if algorithm == 'numerical':
        # assuming K is a RealSet
        from sage.sets.real_set import InternalRealInterval
        assert(isinstance(K, InternalRealInterval))
        
        xmin = K.lower(); xmax = K.upper()
        
        # assuming x = SR.var('x') or assert(f.variables()[0] in SR)
        if not f.variables()[0] in SR:
            f = sum([SR.var('x')^k * fi for k, fi in enumerate(f.coefficients())])
            
        ans = numerical_integral(abs(f)^q, xmin, xmax)[0]^(1/q)
        
        # alternatives:
        #ans = numerical_integral(exp(q*log(abs(f)), xmin, xmax)[0]^(1/q) 
        #ans = integrate(exp(q*log(abs(f)), xmin, xmax, algorithm='giac')[0]^(1/q)
        
    elif algorithm == 'exact':
        # assuming K is a Polyhedron
        #from sage.geometry.polyhedron.base import Polyhedron
        #assert(isinstance(K, Polyhedron)) # wrong: should be a derived class
        
        # since we cannot do abs(f), we restrict to integer and even q
        if not q.is_integer():
            raise ValueError("The exact algorithm requires integer q")
        if not q.mod(2) == 0:
            raise ValueError("The exact algorithm requires even q")
            
        # assuming x = polygen(QQ, 'x')
        ans = K.integrate(f^q)^(1/q)
        
    else:
        raise ValueError("Algorithm not recognised. Use numerical or exact.")
    
    return ans

In [413]:
# Example
x = polygen(QQ, 'x')
norm_q(2-x+x^4+x^2, q=2, algorithm='exact')

1/3*sqrt(533/14)

In [414]:
_.n()

2.05673496838040

In [416]:
x = var('x')
norm_q(2-x+x^4+x^2, q=2, algorithm='numerical')

2.0567349683804013

#### Testing the previous constructions with concrete examples

Consider a 2nd degree polynomial,

In [421]:
# independent variable
x, = polygens(SR, 'x')

# coefficients (symbolic)
f0, f1, f2 = SR.var(['f0', 'f1', 'f2'])

f = f0 + f1*x + f2*x^2
f

f2*x^2 + f1*x + f0

In $\K = [0, 1]$, we can compute $\Vert f \Vert_q$ for different values of $q$. We are able to use the exact computation for integer and even exponent.

In [489]:
fev = f(f0=1, f1=2, f2=3)
fev

3*x^2 + 2*x + 1

In [490]:
pretty_print(LatexExpr(r"f := "), fev)

for q in [1, 3/2, 2, 5/2, 3]:
    if q.is_integer() and q.mod(2) == 0: 
        norm_qi = norm_q(fev, q=q, algorithm='exact')
        pretty_print( "q = ", q, LatexExpr(r"\qquad \Vert f \Vert_q = "), norm_qi, \
                     LatexExpr(r"\approx "), N(norm_qi))
    else:
        norm_qi = norm_q(fev, q=q, algorithm='numerical')
        pretty_print( "q = ", q, LatexExpr(r"\qquad \Vert f \Vert_q = "), norm_qi )
    

f :=  3*x^2 + 2*x + 1

'q = ' 1 \qquad \Vert f \Vert_q =  3.0000000000000004

'q = ' 3/2 \qquad \Vert f \Vert_q =  3.174133608878121

'q = ' 2 \qquad \Vert f \Vert_q =  sqrt(167/15) \approx  3.33666500166459

'q = ' 5/2 \qquad \Vert f \Vert_q =  3.4855638990375124

'q = ' 3 \qquad \Vert f \Vert_q =  3.6204886975210298

In [491]:
pretty_print(LatexExpr(r"f := "), fev)

# d should be at least as the degree of f
dmin = 1+f.degree()

for q in [1, 3/2, 2, 5/2, 3]:
    
    if q.is_integer():
        qmoments = moments_Zq(d=dmin, q=q, algorithm='exact')
        ans = Ly(qmoments, fev, abs_elementwise=True)
        pretty_print( "q = ", q, LatexExpr(r"\qquad \sum_\alpha |f_\alpha| z_{\alpha, q} = "), ans, \
                     LatexExpr(r"\approx "), N(ans))
    else:
        ans = norm_q(fev, q=q, algorithm='numerical')
        pretty_print( "q = ", q, LatexExpr(r"\qquad \sum_\alpha |f_\alpha| z_{\alpha, q} = "), ans )
    

f :=  3*x^2 + 2*x + 1

'q = ' 1 \qquad \sum_\alpha |f_\alpha| z_{\alpha, q} =  3 \approx  3.00000000000000

'q = ' 3/2 \qquad \sum_\alpha |f_\alpha| z_{\alpha, q} =  3.174133608878121

'q = ' 2 \qquad \sum_\alpha |f_\alpha| z_{\alpha, q} =  2*sqrt(1/3) + 3*sqrt(1/5) + 1 \approx  3.49634132487913

'q = ' 5/2 \qquad \sum_\alpha |f_\alpha| z_{\alpha, q} =  3.4855638990375124

'q = ' 3 \qquad \sum_\alpha |f_\alpha| z_{\alpha, q} =  2*(1/4)^(1/3) + 3*(1/7)^(1/3) + 1 \approx  3.82819492561900

By the triangle inequality, one has that:
$$
\Vert f \Vert_q  = \left\Vert \sum_\alpha f_\alpha x^\alpha \right\Vert_q\leq \sum_{\alpha} |f_\alpha| z_{\alpha, q}.
$$

In some concrete examples:

In $\K = [0, 1]$ and with $q=1$,

a. $\Vert 3x^2 - 2x + 1\Vert_1 = 1 < \sum_{\alpha} |f_\alpha| z_{\alpha, 1} = 3$.

b. $\Vert 3x^2 + 2x + 1\Vert_1 = 3 = \sum_{\alpha} |f_\alpha| z_{\alpha, 1} = 3$.

This concludes the presentation of moment sequence computation using SageMath.

**Perspectives : **

- computational representation of an infinite sequence (moments sequence)
- moments matrix and localizing matrix
- interaction with the semidefinite programming interface